<a href="https://colab.research.google.com/github/davidharrisnet/genai/blob/main/llama2_7b_chat_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --upgrade pip --quiet

In [1]:
!pip install transformers accelerate einops  xformers bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
import os
from torch import cuda, bfloat16
from time import time
import transformers
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
model_id = os.path.join("/content","drive","My Drive", "models", "Llama-2-7b-chat-hf", "snapshots", "1")

In [ ]:
os.path.exists(model_id)


True

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(model.get_memory_footprint())

7067942912


In [ ]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 2.962 sec.


In [ ]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=2000,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [ ]:
questions = [ "Write me a detailed scenario for a potential Russian invasion of Moldova.",

"Now identify one person in this scenario who would be a potential HUMINT source for the US Intelligence Community and write a short source dossier. Be sure to include their placement and access.",
"Write me three short intelligence information reports that came from that source about various events in the scenario.",
"Add dates and times for the intelligence reports." ]




In [ ]:
test_model(tokenizer,
           query_pipeline,
           questions[0])


KeyboardInterrupt: ignored

In [ ]:
test_model(tokenizer,
           query_pipeline,
           questions[1])


In [ ]:
test_model(tokenizer,
           query_pipeline,
           questions[2])


In [ ]:
test_model(tokenizer,
           query_pipeline,
           questions[3])


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
pip install langchain[all] --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
xformers 0.0.23 requires torch==2.1.1, but you have torch 1.13.1 which is incompatible.


In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer
from langchain.chains import ConversationChain
from langchain import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.memory import  ConversationSummaryMemory

In [ ]:
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=10000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

In [ ]:
prompt_template = """<s>[INST] <<SYS>>
{{ You are an intelligence analyst}}<<SYS>>
###

Previous Conversation:
'''
{history}
'''

{{{input}}}[/INST]

"""
prompt  = PromptTemplate(template=prompt_template, input_variables=['input', 'history'])

In [ ]:
#memory = ConversationSummaryMemory(llm=llm)
memory = ConversationBufferWindowMemory(k=10)

chain = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.run(questions[0])

"As an intelligence analyst, I must first emphasize that predicting the actions of a sovereign state is a complex and challenging task. However, based on my knowledge of the current geopolitical landscape and historical patterns of Russian behavior, I can provide a plausible scenario for a potential Russian invasion of Moldova.\n\nScenario:\n\nThe situation in Moldova has been escalating over the past year, with tensions between the government in Chisinau and the breakaway region of Transnistria reaching a boiling point. The region, which has been under the de facto control of Russian troops since the early 1990s, has been demanding greater autonomy and recognition as a separate entity from the Moldovan government.\n\nIn recent months, the situation has deteriorated further, with both sides engaging in a series of provocative actions. The Moldovan government has been attempting to assert its authority over Transnistria, including through the use of force, while the Russian military has

In [ ]:
chain.run(questions[1])

"Source Dossier:\n\nName: Alexei Petrov\n\nPlacement and Access:\n\nAlexei Petrov is a 45-year-old former Moldovan military officer who has been living in the breakaway region of Transnistria for the past decade. Petrov served as a colonel in the Moldovan military before retiring in 2010 due to a leg injury. He has since become a vocal critic of the Moldovan government and its handling of the Transnistria conflict.\n\nPetrov currently resides in the city of Tiraspol, the administrative center of Transnistria. He has established himself as a local leader among the ethnic Russian population, and has a reputation for being well-connected and knowledgeable about the region's political landscape.\n\nPetrov's access to information and resources would be invaluable to the US Intelligence Community. As a former military officer and current resident of Transnistria, he has extensive connections within the region's military and political elites. He is also well-versed in the history of the Trans

In [ ]:
chain.run(questions[2])

"---\n\n\n**Intelligence Information Report 1:**\n\nTitle: Increased Russian Military Presence in Transnistria\n\nSource: Alexei Petrov\n\nDate: March 10, 2023\n\nSummary: There has been a significant increase in Russian military presence in Transnistria in recent weeks, according to sources in the region. The Russian military has deployed additional troops and equipment to the region, including armored vehicles, artillery, and fighter jets. This buildup has been accompanied by a heightened rhetoric from Russian officials, with some calling for the protection of ethnic Russians in Moldova.\n\n**Intelligence Information Report 2:**\n\nTitle: Moldovan Government in Disarray\n\nSource: Alexei Petrov\n\nDate: March 15, 2023\n\nSummary: The Moldovan government has been plagued by infighting and ineffectiveness in recent weeks, according to sources in Chisinau. Key ministers have resigned their positions, and the government has been unable to pass important legislation. This power vacuum has

In [ ]:
chain.run(questions[3])

"Intelligence Information Report 1: Increased Russian Military Presence in Transnistria\n\nDate: March 10, 2023\nTime: 10:00 AM\n\nSource: Alexei Petrov\n\nSummary: According to Alexei Petrov, there has been a significant increase in Russian military presence in Transnistria in recent weeks. Russian troops and equipment have been deployed to the region, including armored vehicles, artillery, and fighter jets. This buildup has been accompanied by a heightened rhetoric from Russian officials, with some calling for the protection of ethnic Russians in Moldova.\n\nIntelligence Information Report 2: Moldovan Government in Disarray\n\nDate: March 15, 2023\nTime: 2:00 PM\n\nSource: Alexei Petrov\n\n\nSummary: According to Alexei Petrov, the Moldovan government has been plagued by infighting and ineffectiveness in recent weeks. Key ministers have resigned their positions, and the government has been unable to pass important legislation. This power vacuum has allowed the Russian military to mak

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages.base import BaseMessage

In [ ]:
class FileChatMessageHistory(BaseChatMessageHistory):
    storage_path:  str
    session_id: str

    @property
    def messages(self):
        with open(os.path.join(storage_path, session_id), 'r:utf-8') as f:
            messages = json.loads(f.read())
            return messages_from_dict(messages)

    def add_message(self, message: BaseMessage) -> None:
        messages = self.messages.append(_message_to_dict(message))
        with open(os.path.join(storage_path, session_id), 'w') as f:
            json.dump(f, messages)

    def clear(self):
        with open(os.path.join(storage_path, session_id), 'w') as f:
            f.write("[]")